# Goals of this notebook

In this notebook, we will import the background and foreground LCA databases. We will also create the biosphere database.

# Importing packages

In [1]:
import bw2data as bd
import bw2io as bi

In [2]:
assert bd.__version__ > (4, 0), "This notebook is only for Brightway 2.5"

In [3]:
bd.projects.set_current("LC IMPACT case study")

# Create the `ecoinvent 3.8` database with the `cutoff` system model and the associated `biosphere` database

In [9]:
from ecoinvent_interface import EcoinventRelease, Settings, ReleaseType, CachedStorage

# Enter your identifiers for Ecoquery
username = ""
password = ""

ei_version = '3.8'
system_model = 'cutoff'

my_settings = Settings(username=username, password=password)

In [10]:
ei = EcoinventRelease(my_settings)

In [11]:
ei.list_versions()

['3.10.1',
 '3.10',
 '3.9.1',
 '3.9',
 '3.8',
 '3.7.1',
 '3.7',
 '3.6',
 '3.5',
 '3.4',
 '3.3',
 '3.2',
 '3.1',
 '3.01',
 '2']

In [12]:
ei.list_system_models(ei_version)

['cutoff', 'consequential', 'apos']

In [13]:
ei.list_system_models(ei_version, translate=False)

['Allocation cut-off by classification',
 'Substitution, consequential, long-term',
 'Allocation at the Point of Substitution']

In [14]:
ei.get_release(version=ei_version, system_model=system_model, release_type=ReleaseType.ecospold)

WindowsPath('C:/Users/rogyn/AppData/Local/pylca/EcoinventInterface/cache/ecoinvent 3.8_cutoff_ecoSpold02')

In [15]:
cs = CachedStorage()
list(cs.catalogue)

['ecoinvent 3.10_cutoff_ecoSpold02.7z',
 'ecoinvent 3.10_LCIA_implementation.7z',
 'ecoinvent 3.10.1_cutoff_ecoSpold02.7z',
 'ecoinvent 3.10.1_LCIA_implementation.7z',
 'ecoinvent 3.8_cutoff_ecoSpold02.7z',
 'ecoinvent 3.8_LCIA_implementation.7z']

In [16]:
bi.import_ecoinvent_release(ei_version, system_model, username, password,
                            biosphere_name='biosphere3')

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds
4421 datasets
	0 exchanges
	0 unlinked exchanges (0 unique)
	


100%|███████████████████████████████████████████████████████████████████████████| 4421/4421 [00:00<00:00, 13270.93it/s]


Vacuuming database 
Created database: biosphere3
Extracting XML data from 19565 datasets
Extracted 19565 datasets in 82.57 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably

100%|███████████████████████████████████████████████████████████████████████████| 19565/19565 [01:41<00:00, 192.16it/s]


Vacuuming database 
Created database: ecoinvent-3.8-cutoff
Substituting Chlortoluron for Chlorotoluron
Skipping unmatched flow Cyfluthrin:(soil, agricultural)
Substituting Thiophanate-methyl for Thiophanat-methyl
Substituting Fluorochloridone for Flurochloridone


# Create database for our functional unit

We now define our functional unit. We will look at one passenger kilometer of transport in a modern auto (EURO 5 standard), with two fuel possibilities:

* Low-sulfur petrol
* E85 from Brazilian sugarcane

The functional unit and the foreground LCI database are available in the separate spreadsheet `"ethanol-inventory.xlsx"`, which is available in this tutorial.

<u> Hypothesis :

Metastudies ([1](http://doi.org/10.1021/es305209a                        ),[2](http://doi.org/10.1021/es502495u                        )) on the effective substitution ratio of ethanol compared to petrol are inconclusive. We therefore assume that ethanol substitutes 1:1 for petrol on an energetic basis in modern, fuel-injected cars.

Ethanol has a lower energy density than petrol, therefor *1* kg of petrol is equivalent to *1.46* kg of ethanol

Emissions from the combustion of ethanol differ from low-sulfur petrol. However, this difference is difficult to summarize, as it is dependent on, among other things, engine type, driving patterns, and
climate ([3](http://doi.org/10.1016/j.rser.2013.03.046                 ), [4](http://doi.org/10.1016/j.rser.2013.09.022                 ), [5](http://doi.org/10.1177/0954407011407254                 )). Given this uncertainty, we make the following rough estimates:

* Ethanol has effectively no sulfur, so no SO2 is emitted.
* Emissions of CO decrease 10 percent
* Hydrocarbon emissions decrease 10 percent

In [21]:
imp = bi.ExcelImporter("ethanol-inventory.xlsx")
imp.apply_strategies()
imp.match_database("ecoinvent-3.8-cutoff", fields=('name', 'unit', 'location'))
imp.match_database(fields=['name'])
assert imp.all_linked

C:\Users\rogyn\AppData\Local\anaconda3\envs\brightway_regional_2\lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Extracted 1 worksheets in 0.41 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 21.29 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
6 datasets
	70 exchanges
	0 unlinked exchanges (0 unique)
	


In [22]:
imp.write_database()

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 546.94it/s]

Vacuuming database 


Created database: Fuel comparison
